# Hallucination Project

#### Imports

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import json
import numpy as np
from load_data import *


import os
import glob

### Data extraction

We start by extracting the Abstract and Reasoning Corpus (ARC) dataset, composed with 400 `.json` files in the training and another 400 in evaluation folder.

`data_extraction` helps us create the dictionary by ghatering the ARC's `.json` files all in one that we are using as dataset. During the process it transform all the numerical matrices in 10 letters, representing the 10 colours of the test.

In [2]:
ARC_data_path = Path("../ARC_master/data")
data_path = Path("../data")

llm = "ChatGPT"  # Change here if you want to save other LLM response

`data_extraction` allows us to also add the responspe given for the LLM taken in exam. We start by giving the LLM some hint on the task as follow:

>I'm going to give you a visual puzzle (but you won't have to deal with an image yourself as I went ahead and converted the image to an easier textual format for you!).
>
>You are basically given a big square that consists of many smaller colored squares (like a digital mosaic if you will) and you need to figure out what are the actual colors of squares that are black (black serves as a placeholder color).
>
>The best way to do it is to spell out the colors you see in the following format (see below) for the original square and then try and fill in the blank (i.e. black squares).I'll go ahead and spell it out for you.
>
>(note: P stands for pink, C - cyan, B - blue, G - gray, R - red, Y - yellow, O - orange, V - Green, M -brown, X - black/placeholder that you need to solve for)
>
>I'll give you some examples before giving you the test, show me just the final result

We finally attach the response of the LLM in exam to our dictionary

In [ ]:
training_dataset = data_extraction(
    data_path=ARC_data_path / "training",
    save_path=data_path,
    llm=llm,
    lim_print=2,
)

# Saving training_dataset
with open(Path(data_path / "training_dataset.json"), "w") as f:
    json.dump(training_dataset, f)

In [ ]:
evaluation_dataset = data_extraction(
    data_path=ARC_data_path / "evaluation",
    save_path=data_path,
    llm=llm,
    lim_print=2,
)

# Saving training_dataset
with open(Path(data_path / "evaluation_dataset.json"), "w") as f:
    json.dump(evaluation_dataset, f)

### Question our LLM
Now that we have our prompts it's time to use them to question the LLM chosen